Data

In [1]:
import torch.utils.data as data
from os import listdir
from os.path import join
from PIL import Image
from os.path import exists, join, basename
from os import remove
from six.moves import urllib
import tarfile
from torchvision.transforms import Compose, CenterCrop, ToTensor, Scale

In [2]:
def is_image_file(filename):
    return any(filename.endswith(extension) for extension in [".png", ".jpg", ".jpeg"])
def load_img(filepath):
    img = Image.open(filepath).convert('YCbCr')
    y, _, _ = img.split()
    return y

In [3]:
class DatasetFromFolder(data.Dataset):
    def __init__(self, image_dir, input_transform=None, target_transform=None):
        super(DatasetFromFolder, self).__init__()
        self.image_filenames = [join(image_dir, x) for x in listdir(image_dir) if is_image_file(x)]

        self.input_transform = input_transform
        self.target_transform = target_transform
    def __getitem__(self, index):
        input_image = load_img(self.image_filenames[index])
        target = input_image.copy()
        if self.input_transform:
            input_image = self.input_transform(input_image)
        if self.target_transform:
            target = self.target_transform(target)
        return input_image, target
    def __len__(self):
        return len(self.image_filenames)

In [4]:
def download_bsd300(dest="./dataset"):
    output_image_dir = join(dest, "BSDS300/images")

    if not exists(output_image_dir):
        url = "http://www2.eecs.berkeley.edu/Research/Projects/CS/vision/bsds/BSDS300-images.tgz"
        print("downloading url ", url)

        data = urllib.request.urlopen(url)

        file_path = join(dest, basename(url))
        with open(file_path, 'wb') as f:
            f.write(data.read())

        print("Extracting data")
        with tarfile.open(file_path) as tar:
            for item in tar:
                tar.extract(item, dest)

        remove(file_path)

    return output_image_dir


def calculate_valid_crop_size(crop_size, upscale_factor):
    return crop_size - (crop_size % upscale_factor)


def input_transform(crop_size, upscale_factor):
    return Compose([
        CenterCrop(crop_size),
        Scale(crop_size // upscale_factor),
        ToTensor(),
    ])


def target_transform(crop_size):
    return Compose([
        CenterCrop(crop_size),
        ToTensor(),
    ])


def get_training_set(upscale_factor):
    root_dir = download_bsd300()
    train_dir = join(root_dir, "train")
    crop_size = calculate_valid_crop_size(256, upscale_factor)

    return DatasetFromFolder(train_dir,
                             input_transform=input_transform(crop_size, upscale_factor),
                             target_transform=target_transform(crop_size))


def get_test_set(upscale_factor):
    root_dir = download_bsd300()
    test_dir = join(root_dir, "test")
    crop_size = calculate_valid_crop_size(256, upscale_factor)

    return DatasetFromFolder(test_dir,
                             input_transform=input_transform(crop_size, upscale_factor),
                             target_transform=target_transform(crop_size))

model

In [5]:
import torch.nn as nn
import torch.nn.init as init


class Net(nn.Module):
    def __init__(self, upscale_factor):
        super(Net, self).__init__()

        self.relu = nn.ReLU()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=5, stride=1, padding=2)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 32, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(32, upscale_factor ** 2, kernel_size=3, stride=1, padding=1)
        self.pixel_shuffle = nn.PixelShuffle(upscale_factor)

        self._initialize_weights()

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.conv3(x)
        x = self.relu(x)
        x = self.conv4(x)
        x = self.pixel_shuffle(x)
        return x

    def _initialize_weights(self):
        init.orthogonal(self.conv1.weight, init.calculate_gain('relu'))
        init.orthogonal(self.conv2.weight, init.calculate_gain('relu'))
        init.orthogonal(self.conv3.weight, init.calculate_gain('relu'))
        init.orthogonal(self.conv4.weight)

train_and_test

In [6]:
import sys
import time
TOTAL_BAR_LENGTH = 80
LAST_T = time.time()
BEGIN_T = LAST_T
def progress_bar(current, total, msg=None):
    global LAST_T, BEGIN_T
    if current == 0:
        BEGIN_T = time.time()  # Reset for new bar.

    current_len = int(TOTAL_BAR_LENGTH * (current + 1) / total)
    rest_len = int(TOTAL_BAR_LENGTH - current_len) - 1

    sys.stdout.write(' %d/%d' % (current + 1, total))
    sys.stdout.write(' [')
    for i in range(current_len):
        sys.stdout.write('=')
    sys.stdout.write('>')
    for i in range(rest_len):
        sys.stdout.write('.')
    sys.stdout.write(']')

    current_time = time.time()
    step_time = current_time - LAST_T
    LAST_T = current_time
    total_time = current_time - BEGIN_T

    time_used = '  Step: %s' % format_time(step_time)
    time_used += ' | Tot: %s' % format_time(total_time)
    if msg:
        time_used += ' | ' + msg

    msg = time_used
    sys.stdout.write(msg)

    if current < total - 1:
        sys.stdout.write('\r')
    else:
        sys.stdout.write('\n')
    sys.stdout.flush()
# return the formatted time
def format_time(seconds):
    days = int(seconds / 3600/24)
    seconds = seconds - days*3600*24
    hours = int(seconds / 3600)
    seconds = seconds - hours*3600
    minutes = int(seconds / 60)
    seconds = seconds - minutes*60
    seconds_final = int(seconds)
    seconds = seconds - seconds_final
    millis = int(seconds*1000)
    output = ''
    time_index = 1
    if days > 0:
        output += str(days) + 'D'
        time_index += 1
    if hours > 0 and time_index <= 2:
        output += str(hours) + 'h'
        time_index += 1
    if minutes > 0 and time_index <= 2:
        output += str(minutes) + 'm'
        time_index += 1
    if seconds_final > 0 and time_index <= 2:
        output += str(seconds_final) + 's'
        time_index += 1
    if millis > 0 and time_index <= 2:
        output += str(millis) + 'ms'
        time_index += 1
    if output == '':
        output = '0ms'
    return output

In [7]:
from __future__ import print_function
from math import log10

import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
#from SubPixelCNN.model import Net
#from misc import progress_bar
from torch.optim import lr_scheduler

In [8]:
class SubPixelTrainer(object):
    def __init__(self, training_loader, testing_loader):
        self.model = None
        self.lr = 0.01
        self.nEpochs = 25
        self.criterion = None
        self.optimizer = None
        self.scheduler = None
        #elf.GPU_IN_USE = torch.cuda.is_available()
        self.seed = 111
        self.upscale_factor = 4
        self.training_loader = training_loader
        self.testing_loader = testing_loader

    def build_model(self):
        self.model = Net(upscale_factor=self.upscale_factor)
        self.criterion = nn.MSELoss()
        torch.manual_seed(self.seed)

        #if self.GPU_IN_USE:
            #torch.cuda.manual_seed(self.seed)
            #self.model.cuda()
            #cudnn.benchmark = True

        self.optimizer = optim.Adam(self.model.parameters(), lr=self.lr)
        self.scheduler = lr_scheduler.MultiStepLR(self.optimizer, milestones=[50, 75, 100], gamma=0.5)  # lr decay

    def save(self):
        model_out_path = "sub_pixel_model_path.pth"
        torch.save(self.model, model_out_path)
        print("Checkpoint saved to {}".format(model_out_path))

    def train(self):
        """
        data: [torch.cuda.FloatTensor], 4 batches: [64, 64, 64, 8]
        """
        self.model.train()
        train_loss = 0
        for batch_num, (data, target) in enumerate(self.training_loader):
            #if self.GPU_IN_USE:
                #data, target = Variable(data).cuda(), Variable(target).cuda()
            data, target = Variable(data), Variable(target)
            self.optimizer.zero_grad()
            loss = self.criterion(self.model(data), target)
            train_loss += loss.data[0]
            loss.backward()
            self.optimizer.step()
            progress_bar(batch_num, len(self.training_loader), 'Loss: %.4f' % (train_loss / (batch_num + 1)))

        print("    Average Loss: {:.4f}".format(train_loss / len(self.training_loader)))

    def test(self):
        """
        data: [torch.cuda.FloatTensor], 10 batches: [10, 10, 10, 10, 10, 10, 10, 10, 10, 10]
        """
        self.model.eval()
        avg_psnr = 0
        for batch_num, (data, target) in enumerate(self.testing_loader):
            #if self.GPU_IN_USE:
                #data, target = Variable(data).cuda(), Variable(target).cuda()
            data, target = Variable(data), Variable(target)
            prediction = self.model(data)
            mse = self.criterion(prediction, target)
            psnr = 10 * log10(1 / mse.data[0])
            avg_psnr += psnr
            progress_bar(batch_num, len(self.testing_loader), 'PSNR: %.4f' % (avg_psnr / (batch_num + 1)))

        print("    Average PSNR: {:.4f} dB".format(avg_psnr / len(self.testing_loader)))

    def validate(self):
        self.build_model()
        for epoch in range(1, self.nEpochs + 1):
            print("\n===> Epoch {} starts:".format(epoch))
            self.train()
            self.test()
            self.scheduler.step(epoch)
            if epoch == self.nEpochs:
                self.save()

begin

In [9]:
train_set = get_training_set(4)
test_set = get_test_set(4)
training_data_loader = data.DataLoader(dataset=train_set, batch_size=2, shuffle=True)
testing_data_loader = data.DataLoader(dataset=test_set, batch_size=2, shuffle=False)
model = SubPixelTrainer(training_data_loader, testing_data_loader)
model.validate()


===> Epoch 1 starts:
 100/100 [================================================================================>]  Step: 288ms | Tot: 22s768ms | Loss: 0.0405======================>........................................................]  Step: 201ms | Tot: 6s274ms | Loss: 0.1129 [==========================>.....................................................]  Step: 202ms | Tot: 7s190ms | Loss: 0.1018
    Average Loss: 0.0405
 50/50 [================================================================================>]  Step: 79ms | Tot: 4s799ms | PSNR: 20.3692............]  Step: 88ms | Tot: 1s166ms | PSNR: 20.6730======================================================>.......................]  Step: 112ms | Tot: 3s529ms | PSNR: 20.5033
    Average PSNR: 20.3692 dB

===> Epoch 2 starts:
 100/100 [================================================================================>]  Step: 241ms | Tot: 22s376ms | Loss: 0.0071..............................................]  Step: 197ms | Tot:

 100/100 [================================================================================>]  Step: 237ms | Tot: 23s63ms | Loss: 0.0053=================>..............................................................]  Step: 207ms | Tot: 4s821ms | Loss: 0.0047 [=========================>......................................................]  Step: 201ms | Tot: 7s158ms | Loss: 0.0044 [==========================>.....................................................]  Step: 212ms | Tot: 7s370ms | Loss: 0.0044================>.....................]  Step: 207ms | Tot: 16s804ms | Loss: 0.0046 [============================================================>...................]  Step: 197ms | Tot: 17s226ms | Loss: 0.0047
    Average Loss: 0.0053
 50/50 [================================================================================>]  Step: 88ms | Tot: 4s394ms | PSNR: 21.4060======>..............]  Step: 107ms | Tot: 3s548ms | PSNR: 21.4278======================================================

 100/100 [================================================================================>]  Step: 224ms | Tot: 21s947ms | Loss: 0.0055.................................................]  Step: 196ms | Tot: 630ms | Loss: 0.0040=======================>....................................................]  Step: 205ms | Tot: 7s346ms | Loss: 0.0052================>...................................................]  Step: 198ms | Tot: 7s544ms | Loss: 0.0053=============================>..................................................]  Step: 199ms | Tot: 7s967ms | Loss: 0.0054...................................]  Step: 195ms | Tot: 8s624ms | Loss: 0.0056================>............................................]  Step: 205ms | Tot: 9s469ms | Loss: 0.0056===================================>..........................................]  Step: 204ms | Tot: 10s87ms | Loss: 0.0056 [=======================================================>........................]  Step: 213ms | Tot: 15s76ms

 50/50 [================================================================================>]  Step: 104ms | Tot: 5s164ms | PSNR: 23.5768.......................................................]  Step: 93ms | Tot: 640ms | PSNR: 24.3702 [=================================>..............................................]  Step: 89ms | Tot: 2s93ms | PSNR: 23.8139====================================>....................................]  Step: 108ms | Tot: 2s745ms | PSNR: 23.8695=============================>...............................]  Step: 88ms | Tot: 3s51ms | PSNR: 23.6522=========================================>..............................]  Step: 104ms | Tot: 3s155ms | PSNR: 23.7656========================================================>....................]  Step: 99ms | Tot: 3s794ms | PSNR: 23.7123 [============================================================>...................]  Step: 98ms | Tot: 3s893ms | PSNR: 23.6467=====================================================>....

/home/liar/anaconda3/envs/python3/lib/python3.5/site-packages/torch/serialization.py:147: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
